In [1]:
import sys
# not best practice, but makes it easier to import from subdirectory
sys.path.insert(0, './lib')

import experiment as exp
import model as m
import plot
import predict
import runner2 as r

import numpy as np
import matplotlib.pyplot as plt

import os
import torch

import umap
import matplotlib
from sklearn.preprocessing import StandardScaler

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

_ = plt.rcParams['axes.grid'] = False # turn off grid

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0')

Invoking __init__.py for NDNT.utils


In [32]:
def tconv_scaffold_iter_expanded():
    # Temporal Convolutional Scaffold with Iterative Layer
    tconv_layer = m.TemporalConvolutionalLayer(
        num_filters=2,
        num_inh=1,
        filter_dims=[1, 17, 1, 11],
        window='hamming',
        NLtype=m.NL.relu,
        norm_type=m.Norm.unit,
        bias=False,
        initialize_center=True,
        output_norm='batch',
        padding='spatial',
        reg_vals={'d2xt': 0.0001,
                  'center': 0,
                  'bcs': {'d2xt': 1}})

    proj_layer = m.TemporalConvolutionalLayer(
        num_filters=16,
        num_inh=8,
        filter_dims=[2, 17, 1, 1], # this is because the input coming to this layer is 2D
        NLtype=m.NL.linear,
        bias=False,
        initialize_center=False)

    itert_layer = m.IterativeTemporalConvolutionalLayer(
        num_filters=16,
        num_inh=8,
        filter_dims=7,
        num_lags=2,
        window='hamming',
        NLtype=m.NL.relu,
        norm_type=m.Norm.unit,
        bias=False,
        initialize_center=True,
        output_norm='batch',
        num_iter=3,
        output_config='full',
        reg_vals={'d2xt': 0.0001,
                  'center': 0,
                  'bcs': {'d2xt': 1}})

    readout_layer = m.Layer(
        # because we have inhibitory subunits on the first layer
        pos_constraint=True,
        norm_type=m.Norm.none,
        NLtype=m.NL.softplus,
        bias=True,
        initialize_center=True)

    inp_stim = m.Input(covariate='stim', input_dims=[1,36,1,14])

    core_net = m.Network(layers=[tconv_layer, proj_layer, itert_layer],
                         network_type=m.NetworkType.scaffold,
                         name='core')
    readout_net = m.Network(layers=[readout_layer],
                            name='readout')
    # this is set as a starting point, but updated on each iteration
    output_11 = m.Output(num_neurons=11)

    inp_stim.to(core_net)
    core_net.to(readout_net)
    readout_net.to(output_11)
    itert_model = m.Model(output_11,
                          name='TconvScaffoldIter',
                          create_NDN=True, verbose=True)
    return itert_model

In [33]:
model = tconv_scaffold_iter_expanded()

=== MODEL ===
--- core ---
{'ffnet_n': None,
 'ffnet_type': 'scaffold',
 'layer_list': [{'NLtype': 'relu',
                 'bias': False,
                 'bias_initializer': 'zeros',
                 'conv_dims': None,
                 'dilation': 1,
                 'filter_dims': [1, 17, 1, 11],
                 'folded_lags': False,
                 'initialize_center': True,
                 'input_dims': [1, 36, 1, 14],
                 'layer_type': 'tconv',
                 'norm_type': 1,
                 'num_filters': 2,
                 'num_inh': 1,
                 'output_norm': 'batch',
                 'padding': 'spatial',
                 'pos_constraint': False,
                 'reg_vals': {'bcs': {'d2xt': 1}, 'center': 0, 'd2xt': 0.0001},
                 'res_layer': False,
                 'stride': 1,
                 'temporal_tent_spacing': 1,
                 'weights_initializer': 'xavier_uniform',
                 'window': 'hamming'},
                {'N

In [22]:
e = exp.load('iter_exps07_2iters', experiment_location='experiments', datadir='Mdata')
dataset = e.trials[0].dataset

lazy loading dataset
Loading expt04
  Time embedding...
  Done.
Loading expt06
  Time embedding...
  Done.
Loading expt07
  Time embedding...
  Done.
Loading expt09
  Time embedding...
  Done.
Loading expt11
  Time embedding...
  Done.


In [25]:
model.NDN.list_parameters()

Network 0:
  Layer 0:
      weight: torch.Size([187, 2])
  Layer 1:
      weight: torch.Size([17, 16])
  Layer 2:
      weight: torch.Size([224, 16])
Network 1:
  Layer 0:
      weight: torch.Size([2376, 11])
      bias: torch.Size([11])


In [26]:
model.NDN.networks[0].layers[0].input_dims, model.NDN.networks[0].layers[0].output_dims

([1, 36, 1, 14], [2, 36, 1, 4])

In [27]:
model.NDN.networks[0].layers[1].input_dims, model.NDN.networks[0].layers[1].output_dims

([2, 36, 1, 4], [16, 36, 1, 4])

In [28]:
model.NDN.networks[0].layers[2].input_dims, model.NDN.networks[0].layers[2].output_dims

([16, 36, 1, 4], [48, 36, 1, 1])

In [29]:
model.NDN.networks[1].layers[0].input_dims, model.NDN.networks[0].layers[2].output_dims

([66, 36, 1, 1], [48, 36, 1, 1])

In [30]:
model.NDN(dataset[:10])

x torch.Size([10, 504]) w torch.Size([2, 1, 17, 11]) s torch.Size([10, 1, 52, 14])
x torch.Size([10, 288]) w torch.Size([16, 1, 17, 1]) s torch.Size([10, 2, 52, 4])


RuntimeError: Given groups=1, weight of size [16, 1, 17, 1], expected input[10, 2, 52, 4] to have 1 channels, but got 2 channels instead

In [36]:
a = model.NDN.networks[0].layers[0](dataset[:30]['stim'])
b = model.NDN.networks[0].layers[1](a)
c = model.NDN.networks[0].layers[2](b)
a.shape, b.shape, c.shape

x torch.Size([30, 504]) w torch.Size([2, 1, 17, 11]) s torch.Size([30, 1, 52, 14])
x torch.Size([30, 288]) w torch.Size([16, 2, 17, 1]) s torch.Size([30, 2, 52, 4])


(torch.Size([30, 288]), torch.Size([30, 2304]), torch.Size([30, 1728]))

In [19]:
model.NDN.networks[0].layers[1]._npads

(0, 0, 8, 8)